In [4]:
mb = 1
mw = 1
Ib = 1
Iw = 1
l = 1
d = 1
g = 9.81
def calculate_dynamics(tau, y, dy):
    f = np.array([tau, -tau])
    M = np.array([[Ib + l**2 * mb, np.pi * d * l * mb * np.cos(y[0])], [np.pi * d * l * mb * np.cos(y[0]),  Iw + (np.pi * d)**2 * (mb + mw)]])
    c = np.array([-l * g * mb * np.sin(y[0]), -np.pi * d * y[0]**2 * l * mb * np.sin(y[0])])
    d2y = np.linalg.inv(M).dot(f - c)
    return d2y 

In [ ]:
y0 = np.array([])

In [29]:
from ipywidgets import Image
from time import sleep
from threading import Thread

sprite1 = Image.from_file('sprites/body.png')
sprite2 = Image.from_file('sprites/wheel.png')

canvas1 = Canvas(width=500, height=500)
#canvas1.fill_style = '#a9cafc'
#canvas1.fill_rect(0, 0, 500, 500)
canvas1.draw_image(sprite1, 0, 0)
canvas1.draw_image(sprite2, 100, 100)
canvas1.rotate()

Canvas(width=500)

In [39]:
canvas = Canvas(width=200, height=200)

canvas.fill_rect(50, 25, 100, 100)
canvas.clear_rect(50, 45, 60, 60)
#canvas.stroke_rect(50, 50, 50, 50)

canvas

Canvas(height=200, width=200)

In [1]:
import ipycanvas as ipc

In [6]:
# how many pixels one meter corresponds to, defines the scale
meter_in_pixels = 100
# convert meters to pixels 
def in_pixels(distance_in_meters):
    return distance_in_meters * meter_in_pixels

# draw unicycle wheel
def draw_wheel(canvas, pos, phi, r):
    with ipc.hold_canvas(canvas):
        canvas.clear()
        # 2. wheel:
        canvas.translate(pos[0], pos[1])
        # 2.1. rim
        canvas.line_width = 5
        canvas.stroke_arc(0, 0, r, 0, np.pi * 2) 
        # 2.2. mark
        canvas.rotate(phi)
        canvas.fill_arc(0, -r * 3 / 4, 3, 0, np.pi * 2)
        # 2.3. spokes
        canvas.line_width = 1
        canvas.begin_path()
        canvas.move_to(0, -r)
        canvas.line_to(0, r)
        canvas.move_to(-r, 0)
        canvas.line_to(r, 0)
        canvas.rotate(np.pi / 4)
        canvas.move_to(0, -r)
        canvas.line_to(0, r)
        canvas.move_to(-r, 0)
        canvas.line_to(r, 0)
        canvas.stroke()
    
# draw unicycle body
def draw_body(canvas, pos, theta, l):
    with ipc.hold_canvas(canvas):
        canvas.clear()
        # 3. body:
        canvas.translate(pos[0], pos[1])
        # 3.1. frame
        canvas.rotate(theta)
        canvas.fill_style = 'blue'
        canvas.fill_rect(-2.5, 0, 5, -l)
        # 3.2. motor
        canvas.fill_style = 'darkred'
        canvas.fill_arc(0, 0, 7.5, 0, np.pi * 2)
        # 3.3. load
        canvas.fill_style = 'blue'
        canvas.translate(0, -l)
        canvas.fill_rect(-10, -10, 20)

In [19]:
y0 = np.array([0, np.pi/2])
canvas_width = 600
canvas_height = 300
init_pos = np.array([canvas_width / 2, canvas_height * 3 / 4])
wheel_radius = in_pixels(d / 2)

# initialize canvas
multi_canvas = ipc.MultiCanvas(3, width=canvas_width, height=canvas_height)
# initialize background 
# 1. ground:
multi_canvas[0].fill_style = 'gray'
multi_canvas[0].stroke_rect(0, 0, canvas_width, canvas_height)
multi_canvas[0].fill_rect(0, init_pos[1] + wheel_radius, canvas_width, canvas_height - (init_pos[1] + wheel_radius))

draw_wheel(multi_canvas[1], init_pos, 0, in_pixels(d / 2))
draw_body(multi_canvas[2], init_pos, 0, in_pixels(l))

multi_canvas

MultiCanvas(height=300, width=600)

In [10]:
multi_canvas[1].translate(100,0)
multi_canvas

MultiCanvas(height=300, width=600)